In [ ]:
# @title [STEP 1] Instalasi Library
# Menginstall semua library yang dibutuhkan untuk Training & Visualisasi
%pip install transformers datasets accelerate scikit-learn pandas torch matplotlib seaborn

In [ ]:
# @title [STEP 2] Load Data & Split Train-Test
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 1. Load Data
try:
    df = pd.read_csv('labeled_kandidat_spam.csv', sep=';')
    
    # Pastikan format label integer dan komentar string
    df['label'] = df['label'].astype(int)
    df['comment_text'] = df['comment_text'].astype(str)
    
    # 2. Split Data (80% Train, 20% Test)
    # stratify=df['label'] menjaga rasio spam tetap sama
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
    
    print(f"Data Loaded! Total: {len(df)}")
    print(f"Distibution Label: \n{df['label'].value_counts()}")
    print(f"Training Set: {len(train_df)} baris")
    print(f"Testing Set: {len(test_df)} baris")

except FileNotFoundError:
    print("❌ Error: File 'labeled_kandidat_spam.csv' tidak ditemukan.")

In [ ]:
# @title [STEP 3] Tokenisasi Data
from transformers import BertTokenizer
from datasets import Dataset

# 1. Load Tokenizer IndoBERT
PRETRAINED_MODEL = "indobenchmark/indobert-base-p1"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL)

# 2. Fungsi Tokenisasi
def tokenize_function(examples):
    return tokenizer(
        examples["comment_text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# 3. Convert & Map
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

print("⚙️ Sedang melakukan tokenisasi...")
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)
print("✅ Tokenisasi Selesai!")

In [ ]:
# @title [STEP 4] Setup Model & Metrics
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

# 1. Load Model
model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL,
    num_labels=2
)

# 2. Fungsi Metrik
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

print("✅ Model IndoBERT siap dilatih!")

In [ ]:
# @title [STEP 5] Eksekusi Training (Manual Settings - 15 Epochs)
# Konfigurasi ini terbukti mencapai akurasi ~90% pada dataset ini
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,             # 15 Epochs untuk konvergensi maksimal
    per_device_train_batch_size=8,   # Batch kecil untuk stabilitas
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    learning_rate=3e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

print("🚀 MEMULAI TRAINING...")
trainer.train()
print("🎉 TRAINING SELESAI!")

In [ ]:
# @title [STEP 6] Evaluasi & Save Model
print("📊 Mengevaluasi Model pada Data Test...")
results = trainer.evaluate()

print("\n" + "="*30)
print("HASIL AKHIR:")
print("="*30)
print(f"🎯 Accuracy  : {results['eval_accuracy']:.4f}")
print(f"⭐ F1-Score  : {results['eval_f1']:.4f}")
print(f"🎯 Precision : {results['eval_precision']:.4f}")
print(f"📡 Recall    : {results['eval_recall']:.4f}")
print("="*30)

# Simpan Model
if results['eval_accuracy'] >= 0.85:
    model.save_pretrained("./indobert-spam-detection-final")
    tokenizer.save_pretrained("./indobert-spam-detection-final")
    print("✅ Model tersimpan di './indobert-spam-detection-final'")
else:
    print("⚠️ Akurasi belum 85%, pertimbangkan training ulang.")

In [ ]:
# @title [STEP 7] Visualisasi Confusion Matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import torch
import torch.nn.functional as F

# Prediksi ulang untuk data test
predictions = []
true_labels = []

for index, row in test_df.iterrows():
    text = row['comment_text']
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        pred_idx = torch.argmax(outputs.logits).item()
        
    predictions.append(pred_idx)
    true_labels.append(row['label'])

# Plot
plt.figure(figsize=(6, 5))
cm = confusion_matrix(true_labels, predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Aman", "Spam"], yticklabels=["Aman", "Spam"])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# @title [STEP 8] Visualisasi 3D Embeddings (PCA)
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

# Extract Embeddings (CLS Token)
embeddings = []
labels = []

print("Extracting embeddings for 3D plot...")
# Kita perlu model base untuk ambil hidden state, tapi model klasifikasi juga punya 'bert' encoder
# Akses encoder BERT dari dalam model klasifikasi
encoder = model.bert

for index, row in test_df.iterrows():
    inputs = tokenizer(row['comment_text'], return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = encoder(**inputs)
        # Last hidden state: [batch, seq_len, hidden]
        cls_emb = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        
    embeddings.append(cls_emb[0])
    labels.append(row['label'])

embeddings = np.array(embeddings)
labels = np.array(labels)

# PCA ke 3 Dimensi
pca = PCA(n_components=3)
reduced = pca.fit_transform(embeddings)

# Plotting
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

colors = ['blue', 'red']
class_names = ['Aman', 'Spam']

for idx in [0, 1]:
    mask = labels == idx
    ax.scatter(
        reduced[mask, 0], reduced[mask, 1], reduced[mask, 2],
        c=colors[idx], label=class_names[idx], s=50, alpha=0.7
    )

ax.set_title('3D Visualization of IndoBERT Embeddings')
ax.legend()
plt.show()

In [ ]:
# @title [STEP 9] Demo Prediksi Manual
def predict_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        output = model(**inputs)
        probs = torch.nn.functional.softmax(output.logits, dim=-1)
        pred_idx = torch.argmax(probs).item()
        conf = probs[0][pred_idx].item()
        
    return "SPAM" if pred_idx == 1 else "AMAN", conf

sample_texts = [
    "Situs slot paling gacor, deposit pulsa tanpa potongan",
    "Wah videonya bermanfaat banget min, makasih ya!",
    "Cek bio untuk menang maxwin hari ini"
]

for txt in sample_texts:
    res, conf = predict_text(txt)
    print(f"'{txt}' -> {res} ({conf:.2%})")